# DeepFabric ile Sentetik Veri Üretimi - Kapsamlı Rehber

Bu notebook, DeepFabric kütüphanesini kullanarak lokal LLM modelleri ile sentetik veri üretimini adım adım göstermektedir.

## DeepFabric Nedir?

DeepFabric, yapay zeka modelleri için yüksek kaliteli eğitim verileri oluşturmak üzere tasarlanmış bir sentetik veri üretim çerçevesidir. NVIDIA, Google ve GitHub mühendisleri tarafından geliştirilmiştir.

### Temel Özellikler:

- **Hiyerarşik Konu Ağacı**: Bir ana konudan alt konular oluşturur ve bunlardan sentetik veriler üretir
- **Çoklu LLM Desteği**: OpenAI, Anthropic, Google, Ollama ve OpenAI-uyumlu API'ler
- **Farklı Veri Tipleri**: Soru-cevap, konuşmalar, araç çağrıları, akıl yürütme zincirleri
- **Esnek Formatlar**: TRL, ChatML, Alpaca, özel formatlar
- **Lokal ve Bulut**: Hem lokal hem de bulut tabanlı modeller

### Nasıl Çalışır?

1. **Konu Ağacı Oluşturma**: Verdiğiniz ana konudan hiyerarşik alt konular üretir
2. **Veri Üretimi**: Her konu için soru-cevap çiftleri, konuşmalar veya örnekler oluşturur
3. **Formatlama**: Üretilen veriyi model eğitimi için uygun formata dönüştürür

---

## 1. Kurulum

DeepFabric'i pip ile kuruyoruz:

In [ ]:
# DeepFabric kütüphanesini kur
!pip install deepfabric -q

In [ ]:
# Gerekli kütüphaneleri import et
import os
import json
from pathlib import Path

## 2. Lokal LLM API Konfigürasyonu

### Seçenek 1: OpenAI-Uyumlu API (vLLM, text-generation-webui, LM Studio vb.)

Eğer lokal Llama modelinizi OpenAI-uyumlu bir API ile çalıştırıyorsanız (örneğin vLLM, text-generation-webui, LM Studio):

```python
# API bilgilerinizi buraya girin
os.environ['OPENAI_API_KEY'] = 'dummy-key'  # Lokal API için genelde gerekli değil ama zorunlu
os.environ['OPENAI_BASE_URL'] = 'http://localhost:8000/v1'  # API URL'iniz
```

### Seçenek 2: Ollama ile

Eğer Ollama kullanıyorsanız:
```bash
# Terminal'de Ollama'yı başlatın
ollama serve
```

In [ ]:
# API KONFİGÜRASYONU - KENDİ BİLGİLERİNİZİ GİRİN

# Seçenek 1: OpenAI-uyumlu lokal API
os.environ['OPENAI_API_KEY'] = 'sk-dummy-key'  # Lokal için dummy key
os.environ['OPENAI_BASE_URL'] = 'http://localhost:8000/v1'  # Kendi API URL'iniz

# Model adı - API'nizde kullandığınız model adı
LOCAL_MODEL_NAME = 'llama-3.1-8b'  # Kendi model adınızı girin

# Eğer Ollama kullanıyorsanız:
# PROVIDER = 'ollama'
# LOCAL_MODEL_NAME = 'llama3.1:8b'

PROVIDER = 'openai'  # OpenAI-uyumlu API için 'openai', Ollama için 'ollama'

print(f"✓ API Konfigürasyonu tamamlandı")
print(f"  Provider: {PROVIDER}")
print(f"  Model: {LOCAL_MODEL_NAME}")
print(f"  Base URL: {os.environ.get('OPENAI_BASE_URL', 'default')}")

## 3. Basit Örnek: CLI ile Sentetik Veri Üretimi

DeepFabric'in en basit kullanımı CLI (komut satırı) üzerinden yapılır:

In [ ]:
# Basit bir sentetik veri seti oluştur
# Bu komut:
# 1. "Python Programlama" konusundan konu ağacı oluşturur (depth=2, degree=2)
# 2. Her konu için 8 veri örneği üretir (num-steps=8)
# 3. Sonuçları 'dataset_basit.jsonl' dosyasına kaydeder

!deepfabric generate \
  --mode tree \
  --provider {PROVIDER} \
  --model {LOCAL_MODEL_NAME} \
  --depth 2 \
  --degree 2 \
  --num-steps 8 \
  --topic-prompt "Python Programlama Temelleri" \
  --dataset-save-as dataset_basit.jsonl

In [ ]:
# Oluşturulan veriyi inceleyelim
if os.path.exists('dataset_basit.jsonl'):
    with open('dataset_basit.jsonl', 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i < 2:  # İlk 2 örneği göster
                data = json.loads(line)
                print(f"\n--- Örnek {i+1} ---")
                print(json.dumps(data, indent=2, ensure_ascii=False))
else:
    print("❌ Veri seti henüz oluşturulmadı")

## 4. YAML Konfigürasyonu ile İleri Seviye Kullanım

Daha fazla kontrol için YAML konfigürasyon dosyası kullanabilirsiniz. Bu yöntemle:
- Prompt'ları özelleştirebilirsiniz
- Farklı parametreler ayarlayabilirsiniz
- Çıktı formatını belirleyebilirsiniz

In [ ]:
# YAML konfigürasyon dosyası oluştur
config_yaml = f"""
# Sistem prompt'u - veri setindeki genel davranışı belirler
dataset_system_prompt: |
  Sen yardımcı bir Türkçe AI asistanısın. Kullanıcılara detaylı ve 
  anlaşılır şekilde yanıt veriyorsun.

# Konu ağacı konfigürasyonu
topic_tree:
  # Ana konu - buradan alt konular türetilecek
  topic_prompt: "Makine Öğrenmesi ve Yapay Zeka"
  
  # LLM provider ayarları
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"
  temperature: 0.7  # Yaratıcılık seviyesi (0.0-1.0)
  
  # Ağaç yapısı
  degree: 3  # Her konudan kaç alt konu türetilebilir
  depth: 2   # Konu ağacının derinliği

# Veri üretimi ayarları
data_engine:
  # Veri üretim prompt'u - üretilecek verilerin stilini belirler
  generation_system_prompt: |
    Sen bir yapay zeka eğitmenisin. Öğrencilere kavramları açık ve 
    anlaşılır şekilde öğretiyorsun. Her soruya detaylı, örnekli ve 
    adım adım yanıtlar veriyorsun.
  
  temperature: 0.6
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"

# Veri seti oluşturma parametreleri
dataset:
  creation:
    num_steps: 12      # Toplam kaç veri örneği üretilecek
    batch_size: 2      # Aynı anda kaç örnek üretilecek
    sys_msg: true      # Sistem mesajı dahil edilsin mi
  
  # Çıktı formatı (opsiyonel)
  # builtin://trl_sft_tools  - HuggingFace TRL için
  # builtin://chatml.py      - ChatML formatı
  # builtin://alpaca.py      - Alpaca formatı
  template: "builtin://chatml.py"
"""

# Config dosyasını kaydet
with open('deepfabric_config.yaml', 'w', encoding='utf-8') as f:
    f.write(config_yaml)

print("✓ Konfigürasyon dosyası oluşturuldu: deepfabric_config.yaml")

In [ ]:
# YAML konfigürasyonu ile veri üret
!deepfabric generate \
  --config deepfabric_config.yaml \
  --dataset-save-as dataset_yaml.jsonl

## 5. TXT Dosyasından Soru-Cevap Üretimi

DeepFabric doğrudan txt dosyasından veri üretmez, ancak txt içeriğini **konu (topic)** olarak kullanabilirsiniz.

### Yaklaşım:
1. Txt dosyanızı okuyun
2. İçeriği topic prompt olarak kullanın
3. DeepFabric bu konudan soru-cevaplar üretir

In [ ]:
# Örnek bir txt dosyası oluştur
ornek_metin = """
Python Veri Yapıları

Python'da temel veri yapıları şunlardır:

1. Liste (List): Sıralı, değiştirilebilir koleksiyon. Örnek: [1, 2, 3, 4]
2. Demet (Tuple): Sıralı, değiştirilemez koleksiyon. Örnek: (1, 2, 3)
3. Sözlük (Dictionary): Anahtar-değer çiftleri. Örnek: {'ad': 'Ali', 'yaş': 25}
4. Küme (Set): Sırasız, benzersiz elemanlar. Örnek: {1, 2, 3, 4}

Listeler en çok kullanılan veri yapısıdır çünkü esnektir ve birçok metoda sahiptir.
"""

with open('ornek_konu.txt', 'w', encoding='utf-8') as f:
    f.write(ornek_metin)

print("✓ Örnek txt dosyası oluşturuldu")

In [ ]:
# Txt dosyasını oku ve içeriği topic olarak kullan
with open('ornek_konu.txt', 'r', encoding='utf-8') as f:
    txt_icerik = f.read()

# Özel prompt ile YAML oluştur
txt_config = f"""
dataset_system_prompt: |
  Sen bir Python programlama eğitmenisin. Öğrencilere veri yapılarını 
  açık ve örneklerle öğretiyorsun.

topic_tree:
  # Txt dosyasının içeriğini konu olarak kullanıyoruz
  topic_prompt: |
    {txt_icerik}
    
    Yukarıdaki konular hakkında detaylı alt başlıklar oluştur.
  
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"
  temperature: 0.7
  degree: 4
  depth: 2

data_engine:
  generation_system_prompt: |
    Python veri yapıları hakkında soru-cevap çiftleri oluştur.
    Her cevap:
    - Açık ve anlaşılır olmalı
    - Kod örnekleri içermeli
    - Pratik kullanım senaryoları göstermeli
  
  temperature: 0.5
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"

dataset:
  creation:
    num_steps: 16
    batch_size: 2
    sys_msg: true
  template: "builtin://chatml.py"
"""

with open('txt_based_config.yaml', 'w', encoding='utf-8') as f:
    f.write(txt_config)

print("✓ Txt tabanlı konfigürasyon oluşturuldu")

In [ ]:
# Txt içeriğinden soru-cevap üret
!deepfabric generate \
  --config txt_based_config.yaml \
  --dataset-save-as dataset_from_txt.jsonl

In [ ]:
# Üretilen soru-cevapları incele
if os.path.exists('dataset_from_txt.jsonl'):
    print("\n📊 TXT DOSYASINDAN ÜRETİLEN SORU-CEVAPLAR:\n")
    print("=" * 80)
    
    with open('dataset_from_txt.jsonl', 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if i < 3:  # İlk 3 örneği göster
                data = json.loads(line)
                print(f"\n🔹 Örnek {i+1}:")
                print("-" * 80)
                
                # Messages varsa güzel formatlayalım
                if 'messages' in data:
                    for msg in data['messages']:
                        role = msg.get('role', 'unknown').upper()
                        content = msg.get('content', '')
                        print(f"\n[{role}]")
                        print(content[:500])  # İlk 500 karakter
                        if len(content) > 500:
                            print("... (devamı var)")
                else:
                    print(json.dumps(data, indent=2, ensure_ascii=False)[:1000])
                
                print("\n" + "=" * 80)
else:
    print("❌ Veri seti henüz oluşturulmadı")

## 6. Farklı Veri Formatları

DeepFabric, üretilen veriyi farklı formatlara dönüştürebilir:

In [ ]:
# Mevcut bir veri setini farklı formata çevir
# ChatML formatına çevir
!deepfabric format \
  --dataset dataset_from_txt.jsonl \
  --format builtin://chatml.py \
  --output dataset_chatml.jsonl

print("\n✓ ChatML formatına dönüştürüldü")

In [ ]:
# Alpaca formatına çevir
!deepfabric format \
  --dataset dataset_from_txt.jsonl \
  --format builtin://alpaca.py \
  --output dataset_alpaca.jsonl

print("\n✓ Alpaca formatına dönüştürüldü")

## 7. Parametrelerin Anlamı ve Özelleştirme

### Konu Ağacı Parametreleri:

- **`depth`**: Konu ağacının derinliği. Örnek: `depth=3` → Ana konu → Alt konu → Alt-alt konu
- **`degree`**: Her konudan kaç alt konu üretilir. Örnek: `degree=4` → Her konudan 4 alt konu
- **`topic_prompt`**: Ana konu. Buradan tüm alt konular türetilir

**Toplam konu sayısı**: `degree^depth` (örn: 3^2 = 9 konu)

### Veri Üretim Parametreleri:

- **`num_steps`**: Toplam kaç veri örneği üretilecek
- **`batch_size`**: Aynı anda kaç örnek üretilecek (paralel işlem)
- **`temperature`**: Yaratıcılık seviyesi (0.0 = deterministik, 1.0 = yaratıcı)
- **`sys_msg`**: Sistem mesajı dahil edilsin mi?

### Prompt'lar:

- **`dataset_system_prompt`**: Genel sistem davranışı
- **`generation_system_prompt`**: Veri üretim stili
- **`topic_prompt`**: Ana konu (txt dosyanızın içeriği de olabilir)

## 8. Özel Senaryolar

### Senaryo 1: Kod Örnekleri Üretimi

In [ ]:
kod_config = f"""
dataset_system_prompt: |
  Sen bir programlama eğitmenisin. Her yanıtta çalışan kod örnekleri veriyorsun.

topic_tree:
  topic_prompt: |
    Python ile veri analizi: Pandas, NumPy ve Matplotlib kullanımı
  
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"
  temperature: 0.6
  degree: 3
  depth: 2

data_engine:
  generation_system_prompt: |
    Veri analizi hakkında soru-cevap üret. Her cevap:
    1. Konsepti açıkla
    2. Kod örneği ver (çalışır kod)
    3. Çıktıyı göster
    4. Kullanım alanlarını açıkla
  
  temperature: 0.4
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"

dataset:
  creation:
    num_steps: 20
    batch_size: 2
    sys_msg: true
"""

with open('kod_ornekleri_config.yaml', 'w', encoding='utf-8') as f:
    f.write(kod_config)

print("✓ Kod örnekleri konfigürasyonu hazır")

### Senaryo 2: Çok Turlu Konuşmalar

In [ ]:
konusma_config = f"""
dataset_system_prompt: |
  Sen yardımcı bir müşteri hizmetleri temsilcisisin. Nazik ve çözüm odaklısın.

topic_tree:
  topic_prompt: |
    E-ticaret müşteri hizmetleri: Sipariş sorunları, iade işlemleri, 
    ürün bilgileri, teslimat takibi
  
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"
  temperature: 0.7
  degree: 4
  depth: 2

data_engine:
  generation_system_prompt: |
    Müşteri-temsilci konuşmaları oluştur. Her konuşma:
    - Gerçekçi olmalı
    - 3-5 mesaj alışverişi içermeli
    - Sorunu çözmeli
    - Profesyonel ama samimi olmalı
  
  temperature: 0.7
  provider: "{PROVIDER}"
  model: "{LOCAL_MODEL_NAME}"

dataset:
  creation:
    num_steps: 15
    batch_size: 1
    sys_msg: true
  template: "builtin://chatml.py"
"""

with open('konusma_config.yaml', 'w', encoding='utf-8') as f:
    f.write(konusma_config)

print("✓ Konuşma konfigürasyonu hazır")

## 9. Veri Setini İnceleme ve İstatistikler

In [ ]:
def veri_seti_analizi(dosya_yolu):
    """
    JSONL formatındaki veri setini analiz eder
    """
    if not os.path.exists(dosya_yolu):
        print(f"❌ Dosya bulunamadı: {dosya_yolu}")
        return
    
    veriler = []
    with open(dosya_yolu, 'r', encoding='utf-8') as f:
        for line in f:
            veriler.append(json.loads(line))
    
    print(f"\n📊 Veri Seti Analizi: {dosya_yolu}")
    print("=" * 80)
    print(f"Toplam örnek sayısı: {len(veriler)}")
    
    # Mesaj sayıları
    if veriler and 'messages' in veriler[0]:
        mesaj_sayilari = [len(v['messages']) for v in veriler if 'messages' in v]
        print(f"Ortalama mesaj sayısı: {sum(mesaj_sayilari) / len(mesaj_sayilari):.1f}")
        print(f"Min mesaj sayısı: {min(mesaj_sayilari)}")
        print(f"Max mesaj sayısı: {max(mesaj_sayilari)}")
    
    # Alan isimleri
    if veriler:
        print(f"\nVeri alanları: {list(veriler[0].keys())}")
    
    # İlk örneği göster
    if veriler:
        print("\n📝 Örnek veri:")
        print("-" * 80)
        print(json.dumps(veriler[0], indent=2, ensure_ascii=False)[:1500])
        print("...")
    
    return veriler

# Örnek kullanım
# veri_seti_analizi('dataset_from_txt.jsonl')

## 10. Model Eğitimi için Veri Hazırlığı

Üretilen veriyi model eğitimi için kullanmak:

In [ ]:
# HuggingFace format için
!deepfabric format \
  --dataset dataset_from_txt.jsonl \
  --format builtin://trl_sft_tools \
  --output dataset_hf_ready.jsonl

print("\n✓ HuggingFace TRL için hazır veri seti oluşturuldu")
print("\nBu veri setini şu şekilde kullanabilirsiniz:")
print("""
from datasets import load_dataset
from trl import SFTTrainer

dataset = load_dataset('json', data_files='dataset_hf_ready.jsonl')
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset['train'],
    ...
)
trainer.train()
""")

## 11. İpuçları ve En İyi Pratikler

### 🎯 Kaliteli Veri Üretimi İçin:

1. **Spesifik Konular Seçin**: 
   - ❌ "Programlama" yerine
   - ✅ "Python'da list comprehension kullanımı ve performans optimizasyonu"

2. **Prompt'ları Özelleştirin**:
   - `generation_system_prompt` ile veri stilini belirleyin
   - Örnekler, açıklamalar, kod blokları gibi isteklerinizi ekleyin

3. **Temperature Ayarları**:
   - Konu oluşturma: 0.7-0.8 (daha yaratıcı)
   - Veri üretimi: 0.4-0.6 (daha tutarlı)
   - Teknik içerik: 0.3-0.5 (daha deterministik)

4. **Küçük Başlayın**:
   - İlk denemede `num_steps=5-10` kullanın
   - Kaliteyi kontrol edin
   - Sonra artırın

5. **Batch Size**:
   - Lokal API: 1-2 (kaynak sınırlı)
   - Güçlü API: 4-8 (daha hızlı)

### 🔧 Sorun Giderme:

**API Bağlantı Hatası:**
```python
# API'nizin çalıştığından emin olun
import requests
response = requests.get(os.environ['OPENAI_BASE_URL'].replace('/v1', '/health'))
print(response.status_code)  # 200 olmalı
```

**Düşük Kaliteli Veri:**
- `generation_system_prompt` daha detaylı yapın
- Temperature'ı düşürün
- Daha güçlü bir model deneyin

**Yavaş Üretim:**
- `batch_size` artırın
- Daha küçük bir model kullanın
- `depth` ve `degree` azaltın

## 12. Özet ve Sonraki Adımlar

### 📝 Öğrendikleriniz:

✅ DeepFabric'i kurdunuz ve yapılandırdınız  
✅ Lokal LLM API ile bağlandınız  
✅ Basit ve gelişmiş veri setleri oluşturdunuz  
✅ Txt dosyasından soru-cevap ürettiniz  
✅ Farklı formatları öğrendiniz  
✅ Prompt'ları özelleştirdiniz  

### 🚀 Sonraki Adımlar:

1. **Kendi Verilerinizi Oluşturun**: 
   - Kendi txt dosyalarınızı kullanın
   - Domain-spesifik konular seçin

2. **Kaliteyi Artırın**:
   - Farklı prompt'lar deneyin
   - Temperature ayarlarını optimize edin
   - Üretilen veriyi gözden geçirin

3. **Model Eğitimi**:
   - Ürettiğiniz veri ile modelinizi fine-tune edin
   - QLoRA veya LoRA kullanın
   - Sonuçları değerlendirin

4. **İleri Seviye**:
   - Özel formatlar yazın
   - Araç çağrıları (tool calling) ekleyin
   - Multi-turn konuşmalar oluşturun

### 📚 Kaynaklar:

- GitHub: https://github.com/lukehinds/deepfabric
- Dokümantasyon: Repository README
- Örnekler: `examples/` klasörü

---

**İyi çalışmalar! 🎉**